<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
path = './data/'
emb_data_train = pd.read_csv('./data/node2vec_v2_train.csv.gz',compression = 'gzip',sep='\t')
emb_data_test = pd.read_csv('./data/node2vec_v2_test.csv.gz',compression = 'gzip',sep='\t')
emb_data_train= emb_data_train.drop(columns=['Unnamed: 0'])
emb_data_test= emb_data_test.drop(columns=['Unnamed: 0'])
emb_data_train.head()

Index(['Unnamed: 0', 'is_dorm', 'is_year', 'year_diff', 'from_high_school',
       'to_high_school', 'from_major', 'to_major', 'is_faculty', 'is_gender',
       'd0', 'd1', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17',
       'd18', 'd19', 'd2', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26',
       'd27', 'd28', 'd29', 'd3', 'd30', 'd31', 'd32', 'd33', 'd34', 'd35',
       'd36', 'd37', 'd38', 'd39', 'd4', 'd40', 'd41', 'd42', 'd43', 'd44',
       'd45', 'd46', 'd47', 'd48', 'd49', 'd5', 'd50', 'd51', 'd52', 'd53',
       'd54', 'd55', 'd56', 'd57', 'd58', 'd59', 'd6', 'd60', 'd61', 'd62',
       'd63', 'd7', 'd8', 'd9', 'label'],
      dtype='object')

,is_dorm,is_year,year_diff,from_high_school,to_high_school,from_major,to_major,is_faculty,is_gender,d0,...,d59,d6,d60,d61,d62,d63,d7,d8,d9,label
0,0,0,1.000000,17819,50093,265,294,1,1,0.586327,...,0.080709,0.663903,0.521325,-1.218457,0.572862,8.867662,0.098672,0.072862,1.136375,0
1,0,1,0.000000,10070,24562,51,60,1,1,1.556291,...,2.682853,-0.022940,-0.371618,-0.987022,1.831474,-0.448014,1.167990,2.401762,0.431279,0
2,0,1,0.000000,1544,6122,14,39,1,1,-0.649554,...,6.857399,-0.348564,2.753245,-0.207652,1.892210,-0.977367,11.321891,2.510515,2.910005,1
3,0,0,2.196375,1894,50410,0,40,0,0,3.757138,...,5.611234,-1.006211,-1.872412,-0.861397,0.114388,-1.096610,-4.892152,-3.294941,0.133597,0
4,0,1,0.000000,3535,21037,238,271,1,1,1.290842,...,0.824280,0.429348,2.083774,0.281935,-1.777484,0.258835,0.234619,-0.287869,2.079238,0


In [27]:
def LogReg(data,test,cols):
    cols.append('label')
    X = data.drop(columns=cols).values
    y = data['label'].values
    X_test = test.drop(columns=cols).values
    y_test = test['label'].values
    scaler = MinMaxScaler(feature_range = (0,1))
    scaler.fit(X)
    #X = scaler.transform(X)
    model = LogisticRegression(C=4.281332398719396, class_weight=None, dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                    random_state=None, solver='liblinear', tol=0.0001, verbose=1,
                    warm_start=False).fit(X, y)
    yhat = model.predict(X_test)
    print(confusion_matrix(y_test, yhat))
    print(classification_report(y_test, yhat))
    print('ROC_AUC_SCORE:',roc_auc_score(y_test, model.decision_function(X_test)))

In [28]:
LogReg(emb_data_train,emb_data_test,[
        ])

[LibLinear][[9298 1074]
 [1546 8705]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88     10372
           1       0.89      0.85      0.87     10251

    accuracy                           0.87     20623
   macro avg       0.87      0.87      0.87     20623
weighted avg       0.87      0.87      0.87     20623

ROC_AUC_SCORE: 0.9390972287823979


In [29]:
pd.concat([emb_data_train,emb_data_test],axis=0)

,is_dorm,is_year,year_diff,from_high_school,to_high_school,from_major,to_major,is_faculty,is_gender,d0,...,d59,d6,d60,d61,d62,d63,d7,d8,d9,label
0,0,0,1.000000,17819,50093,265,294,1,1,0.586327,...,0.080709,0.663903,0.521325,-1.218457,0.572862,8.867662,0.098672,0.072862,1.136375,0
1,0,1,0.000000,10070,24562,51,60,1,1,1.556291,...,2.682853,-0.022940,-0.371618,-0.987022,1.831474,-0.448014,1.167990,2.401762,0.431279,0
2,0,1,0.000000,1544,6122,14,39,1,1,-0.649554,...,6.857399,-0.348564,2.753245,-0.207652,1.892210,-0.977367,11.321891,2.510515,2.910005,1
3,0,0,2.196375,1894,50410,0,40,0,0,3.757138,...,5.611234,-1.006211,-1.872412,-0.861397,0.114388,-1.096610,-4.892152,-3.294941,0.133597,0
4,0,1,0.000000,3535,21037,238,271,1,1,1.290842,...,0.824280,0.429348,2.083774,0.281935,-1.777484,0.258835,0.234619,-0.287869,2.079238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20618,0,1,0.000000,3309,6804,245,268,1,0,0.060135,...,1.569307,0.219338,-0.056801,0.896833,3.806730,0.682038,-1.438232,0.001421,0.016438,1
20619,0,0,1.000000,17454,53475,247,278,1,0,0.719338,...,-1.773060,-0.145525,-1.525118,3.308826,0.035109,0.054840,0.400089,4.942602,-0.257372,1
20620,0,1,0.000000,0,0,0,0,0,0,0.026021,...,0.259740,-0.004445,-0.009570,0.032802,-0.004794,0.001887,-0.039648,0.038391,-0.115764,0
20621,0,0,3.000000,14701,50494,6,6,1,1,-0.169955,...,0.245237,-2.028347,0.000799,2.199125,2.597961,2.013763,-0.012206,0.115874,-8.866046,0
